In [1]:
from sklearn.model_selection import train_test_split
import numpy as np

# For reproducibility
seed = 27
np.random.seed(seed)

raw_data = np.loadtxt("raw_data/train.csv", delimiter = ",", skiprows = 1)

# Split into input (X) and output (Y) variables
X = raw_data[:,1:]
Y = raw_data[:,0]

# Split into 80%% for train and 20% for validation
x_train, x_validate, y_train, y_validate = train_test_split(X, Y, test_size = 0.2, random_state = seed, stratify = Y)


In [2]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 24

# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_validate = x_validate.reshape(x_validate.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_validate = x_validate.reshape(x_validate.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_validate = x_validate.astype('float32')
x_train /= 255
x_validate /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_validate.shape[0], 'validation samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_validate = keras.utils.to_categorical(y_validate, num_classes)


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validate, y_validate))
score = model.evaluate(x_validate, y_validate, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


x_train shape: (33600, 28, 28, 1)
33600 train samples
8400 validation samples
Train on 33600 samples, validate on 8400 samples
Epoch 1/24
33600/33600 [==============================] - 12s 360us/step - loss: 0.4574 - acc: 0.8573 - val_loss: 0.1103 - val_acc: 0.9660
Epoch 2/24
33600/33600 [==============================] - 10s 300us/step - loss: 0.1490 - acc: 0.9572 - val_loss: 0.0745 - val_acc: 0.9757
Epoch 3/24
33600/33600 [==============================] - 10s 300us/step - loss: 0.1124 - acc: 0.9666 - val_loss: 0.0642 - val_acc: 0.9794
Epoch 4/24
33600/33600 [==============================] - 10s 301us/step - loss: 0.0906 - acc: 0.9727 - val_loss: 0.0543 - val_acc: 0.9824
Epoch 5/24
33600/33600 [==============================] - 10s 302us/step - loss: 0.0797 - acc: 0.9757 - val_loss: 0.0517 - val_acc: 0.9830
Epoch 6/24
33600/33600 [==============================] - 10s 300us/step - loss: 0.0723 - acc: 0.9787 - val_loss: 0.0521 - val_acc: 0.9836
Epoch 7/24
33600/33600 [===============